
# Möbius‑Digit‑Thread **Density Grid Explorer (v12)**
Clean rebuilt notebook that reproduces the current best search recipe **from scratch** and
adds two new levers the user asked to probe:

| knob | default | comment |
|------|---------|---------|
| `bright_tail` | 400 | number of leading `1` coefficients |
| `gap_scale` | **20** | divisor applied to prime gaps (dark pulses) |
| `spacing` | **3** | insert a gap after every *spacing* bright pulses |
| `tiny_exp` | 4.5 (α⁻¹/10⁴ˣ) | exponent *k* in α⁻¹/10ᵏ |
| `φ_micro` | 0.003944 ± 1×10⁻⁷ | nine‑point micro grid |
| **NEW** `λ` | 0 … 1.0 | single damped breath `x←(x')^λ` after usual Möbius cycle |
| **NEW** `k_exp` | 3.8 … 4.5 | tiny exponent sweep to pick up second‑order effects |

The notebook prints a summary table sorted by relative error to α and **auto‑highlights**
any row that beats `1 %` (and shouts if one dips below `0.1 %`, as in the v6 reference run).

Run **all** and scroll to the final dataframe. You can then tweak the parameter
lists at the bottom and re‑run cell ⚙️ **Grid sweep** only.


In [1]:

import mpmath as mp, pandas as pd, itertools, re, json, math, uuid
mp.mp.dps = 260  # high precision

ALPHA_INV = mp.mpf('137.035999084')
ALPHA = 1/ALPHA_INV
PHI = (1+mp.sqrt(5))/2


In [2]:

# constant digit strings for  thread-detection
def digits_of(num, n=700):
    s = str(num).replace('.','')[1:]  # drop leading '3.' etc.
    return s[:n]

CONST = {
    'pi': digits_of(mp.pi),
    'e': digits_of(mp.e),
    'phi': digits_of(PHI),
    'gamma': digits_of(mp.euler),
    'sqrt2': digits_of(mp.sqrt(2)),
}


In [3]:

def primes_upto(n=30000):
    sieve = [True]*(n+1)
    sieve[0:2] = [False, False]
    for i in range(2,int(n**0.5)+1):
        if sieve[i]:
            sieve[i*i:n+1:i] = [False]*len(range(i*i, n+1, i))
    return [i for i,v in enumerate(sieve) if v]

def prime_gaps(count, start_idx=0):
    ps = primes_upto(100000)
    return [ps[i+1]-ps[i] for i in range(start_idx, start_idx+count)]


In [4]:

def build_tail(bright_len=400, spacing=3, gap_scale=20):
    gaps = [round(g/gap_scale,4) for g in prime_gaps(bright_len//spacing)]
    tail = []
    gi = iter(gaps)
    for i in range(bright_len):
        tail.append(1)
        if (i+1) % spacing == 0:
            tail.append(next(gi,1))
    return tail

def cf_to_real(coeffs):
    v = mp.mpf(0)
    for a in reversed(coeffs):
        v = 1/(a+v)
    return v


In [5]:
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_fraction(n_bits):
    bits = fib_word_bits(n_bits)
    return mp.nsum(lambda k: int(bits[int(k)])*mp.mpf(2)**(-(int(k)+1)), [0,len(bits)-1])

In [6]:

def breath_once(x, tiny, phi):
    x = phi*x            # φ‑scale
    x -= tiny            # α‑nudge
    x = 1/x              # reciprocal
    x = x/9
    return x

def damped_breath(x, tiny, phi, lam):
    x = breath_once(x, tiny, phi)
    if lam is not None:
        x = x ** lam     # gentle damping
    return x


In [7]:

def const_thread(xdig):
    for name,bank in CONST.items():
        for l in range(3,8):
            sub = xdig[l:l+4]
            if sub in bank: return f'{name}:{sub}'
    return '-'


In [8]:

# === GRID SWEEP ===
phi_centre = mp.mpf('0.003944')
phi_vals = [phi_centre + i*1e-7 for i in range(-4,5)]
k_vals = [3.8,4.0,4.2,4.3,4.4,4.5]
lam_vals = [None,0.2,0.3,0.4,0.5]
gap_scale = 20
spacing = 3

F64  = fib_fraction(64)
F128 = fib_fraction(128)

rows=[]
tail_coeffs = build_tail(spacing=spacing, gap_scale=gap_scale)
seed0 = cf_to_real(tail_coeffs)

for bits in (64,128,256):
    for k in k_vals:
        tiny = ALPHA_INV / (10**k)
        for lam in lam_vals:
            for phi in phi_vals:
                if lam is None:
                    x = breath_once(seed0+tiny, tiny, phi)
                else:
                    x = damped_breath(seed0+tiny, tiny, phi, lam)
                rel = float(abs((x-ALPHA)/ALPHA))
                xdig = str(x)[2:402]
                rows.append(dict(bits=bits,k=k,phi=phi,lam=lam if lam else 0,rel_err=rel,
                                 thread=const_thread(xdig),
                                 fib64_near=abs(x-F64)<1e-5,
                                 fib128_near=abs(x-F128)<1e-5))
df = pd.DataFrame(rows).sort_values('rel_err')
good = df[df['rel_err']<0.01]
print("Rows <1% err:", len(good))
display(df.head(40))


TypeError: string indices must be integers, not 'mpf'